In [1]:
import numpy as np 
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
#from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
import string
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer
import pickle

In [2]:
df = pd.read_csv('FinalTweetList.csv')
df.columns=['tweet','sentiment','bank']
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

In [5]:
#find all banks - first list out all words beginning with @
def find(text):
    list = re.findall("[@]\w+", text)
    return list
df['@'] = (df['tweet']).apply(lambda x: find(x))
df.head()

,tweet,sentiment,bank,@
0,@AxisBankSupport 18..233.... is permanently bu...,displeasure,NaN,[@AxisBankSupport]
1,"@TheOfficialSBI 3 hours in line,still waiting....",displeasure,NaN,[@TheOfficialSBI]
2,@HDFCBank_Cares Both the ATM machines here in ...,displeasure,NaN,"[@HDFCBank_Cares, @sanatm]"
3,@ICICIBank Thanks icici bank I got case in you...,miscellaneous,NaN,"[@ICICIBank, @TheOfficialSBI]"
4,@RBI In Assam all banks are exchanging 4000/- ...,displeasure,NaN,"[@RBI, @TheOfficialSBI]"


In [6]:
#find frequency of words beginning with @
#As we need testing data of 500 values we will only take the banks specified in more than 500 rows in the datset - upon analysing we see that 
#these are - SBI,Axis,HDFC,Kotak and ICICI
lis=df['@'].to_list()
flat = [y for x in lis for y in x]
count = Counter(flat)
count.most_common()

[('@TheOfficialSBI', 2468),
 ('@HDFCBank_Cares', 928),
 ('@ICICIBank_Care', 893),
 ('@KotakBankLtd', 691),
 ('@AxisBankSupport', 562),
 ('@RBI', 481),
 ('@ICICIBank', 448),
 ('@HDFC_Bank', 364),
 ('@arunjaitley', 315),
 ('@narendramodi', 312),
 ('@bankofbaroda', 268),
 ('@AxisBank', 262),
 ('@PMOIndia', 248),
 ('@FinMinIndia', 144),
 ('@udaykotak', 82),
 ('@abndnkmtr', 41),
 ('@YESBANK', 36),
 ('@mdpnb', 27),
 ('@SBICard_Connect', 24),
 ('@ndtv', 23),
 ('@IDBI_Bank', 22),
 ('@BJP4India', 20),
 ('@TimesNow', 19),
 ('@DasShaktikanta', 19),
 ('@mediacrooks', 18),
 ('@suchetadalal', 17),
 ('@chandakochhar', 16),
 ('@aajtak', 16),
 ('@htTweets', 15),
 ('@timesofindia', 14),
 ('@ZeeNews', 14),
 ('@HDFCBankNews', 13),
 ('@ArvindKejriwal', 13),
 ('@CKforCleanIndia', 13),
 ('@theofficialsbi', 13),
 ('@neha_aks', 13),
 ('@CNBCTV18Live', 13),
 ('@MyIndusIndBank', 12),
 ('@karishmakotak', 12),
 ('@OfficeOfRG', 12),
 ('@sudhirchaudhary', 11),
 ('@arjunrammeghwal', 11),
 ('@INCIndia', 11),
 ('@ANI_n

In [7]:
#removing punctuation and converting to lower case
def rmvpuct_lower(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    return text_rc

df['Tweet_str'] = df['tweet'].apply(lambda x:rmvpuct_lower(x))
df.head()

,tweet,sentiment,bank,@,Tweet_str
0,@AxisBankSupport 18..233.... is permanently bu...,displeasure,NaN,[@AxisBankSupport],axisbanksupport is permanently busy amp can...
1,"@TheOfficialSBI 3 hours in line,still waiting....",displeasure,NaN,[@TheOfficialSBI],theofficialsbi hours in linestill waiting wis...
2,@HDFCBank_Cares Both the ATM machines here in ...,displeasure,NaN,"[@HDFCBank_Cares, @sanatm]",hdfcbankcares both the atm machines here in ti...
3,@ICICIBank Thanks icici bank I got case in you...,miscellaneous,NaN,"[@ICICIBank, @TheOfficialSBI]",icicibank thanks icici bank i got case in your...
4,@RBI In Assam all banks are exchanging 4000/- ...,displeasure,NaN,"[@RBI, @TheOfficialSBI]",rbi in assam all banks are exchanging once on...


In [8]:
#finding bank associated with each tweet
def find_bank(text):
    bnk=re.findall(r"hdfc|axis|sbi|icici|kotak",text)
    return bnk
df['bank'] = (df['Tweet_str']).apply(lambda x: find_bank(x))
df.head()

,tweet,sentiment,bank,@,Tweet_str
0,@AxisBankSupport 18..233.... is permanently bu...,displeasure,[axis],[@AxisBankSupport],axisbanksupport is permanently busy amp can...
1,"@TheOfficialSBI 3 hours in line,still waiting....",displeasure,[sbi],[@TheOfficialSBI],theofficialsbi hours in linestill waiting wis...
2,@HDFCBank_Cares Both the ATM machines here in ...,displeasure,[hdfc],"[@HDFCBank_Cares, @sanatm]",hdfcbankcares both the atm machines here in ti...
3,@ICICIBank Thanks icici bank I got case in you...,miscellaneous,"[icici, icici, sbi]","[@ICICIBank, @TheOfficialSBI]",icicibank thanks icici bank i got case in your...
4,@RBI In Assam all banks are exchanging 4000/- ...,displeasure,[sbi],"[@RBI, @TheOfficialSBI]",rbi in assam all banks are exchanging once on...


In [9]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [word for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text
df['Tweet_cleaned'] = df['tweet'].apply(lambda x: clean_text(x))

In [10]:
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df['Tweet_stemmed'] = df['Tweet_cleaned'].apply(lambda x: stemming(x))

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
df['Tweet_lemmatized_stemmed'] = df['Tweet_stemmed'].apply(lambda x: lemmatizer(x))
df.head()

,tweet,sentiment,bank,@,Tweet_str,Tweet_cleaned,Tweet_stemmed,Tweet_lemmatized_stemmed
0,@AxisBankSupport 18..233.... is permanently bu...,displeasure,[axis],[@AxisBankSupport],axisbanksupport is permanently busy amp can...,"[axisbanksupport, permanently, busy, amp, cann...","[axisbanksupport, perman, busi, amp, cannot, g...","[axisbanksupport, perman, busi, amp, cannot, g..."
1,"@TheOfficialSBI 3 hours in line,still waiting....",displeasure,[sbi],[@TheOfficialSBI],theofficialsbi hours in linestill waiting wis...,"[theofficialsbi, hours, linestill, waiting, wi...","[theofficialsbi, hour, linestil, wait, wish, u...","[theofficialsbi, hour, linestil, wait, wish, u..."
2,@HDFCBank_Cares Both the ATM machines here in ...,displeasure,[hdfc],"[@HDFCBank_Cares, @sanatm]",hdfcbankcares both the atm machines here in ti...,"[hdfcbankcares, atm, machines, times, india, n...","[hdfcbankcar, atm, machin, time, india, noida,...","[hdfcbankcar, atm, machin, time, india, noida,..."
3,@ICICIBank Thanks icici bank I got case in you...,miscellaneous,"[icici, icici, sbi]","[@ICICIBank, @TheOfficialSBI]",icicibank thanks icici bank i got case in your...,"[icicibank, thanks, icici, bank, got, case, at...","[icicibank, thank, icici, bank, got, case, atm...","[icicibank, thank, icici, bank, got, case, atm..."
4,@RBI In Assam all banks are exchanging 4000/- ...,displeasure,[sbi],"[@RBI, @TheOfficialSBI]",rbi in assam all banks are exchanging once on...,"[rbi, assam, banks, exchanging, exchanged, yes...","[rbi, assam, bank, exchang, exchang, yesterday...","[rbi, assam, bank, exchang, exchang, yesterday..."


In [11]:
hdfc=df[df.apply(lambda x: 'hdfc' in x['bank'],axis=1)]
axis=df[df.apply(lambda x: 'axis' in x['bank'],axis=1)]
sbi=df[df.apply(lambda x: 'sbi' in x['bank'],axis=1)]
kotak=df[df.apply(lambda x: 'kotak' in x['bank'],axis=1)]
icici=df[df.apply(lambda x: 'icici' in x['bank'],axis=1)]

In [12]:
hdfc_test =hdfc.iloc[0:500]
hdfc_train=hdfc.iloc[500:]
axis_test =axis.iloc[0:500]
axis_train=axis.iloc[500:]
sbi_test =sbi.iloc[0:500]
sbi_train=sbi.iloc[500:]
kotak_test =kotak.iloc[0:500]
kotak_train=kotak.iloc[500:]
icici_test =icici.iloc[0:500]
icici_train=icici.iloc[500:]

In [13]:
train_data = pd.concat([hdfc_train,axis_train,sbi_train,kotak_train,icici_train])

In [14]:
ft_sent=train_data[['Tweet_stemmed','sentiment']]
ft_sent.rename(columns = {'Tweet_stemmed':'features'}, inplace = True) 
x_tr=train_data[['Tweet_str']]
y_tr=train_data[['sentiment']]

C:\Users\KIRHIKAGURUMURTHY\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [15]:
#lis=ft_sent['features'].to_list()
#flat_ft = [y for x in lis for y in x]
# Remove featureList duplicates
#featureList = list(set(flat_ft))
#print(featureList)
def extract_features(tweet):
    tweet_words = set(tweet[0])
    features = {}
    for word in featureList:
        features[tuple(word2vec.wv[word].tolist())] = word in tweet_words
    return (features,tweet[1])
""""def words_features(tweet):
    tweet_words=set(tweet)
    features={}
    features[word]= word in tweet_words
    return features
    
def extract_fts(x,y):
    return[[(word_features(WordPunctTokenizer().tokenize(x[i]),y[i]) for i in range(len(x)))]]

tr=extract_fts(x_tr,y_tr)
MaxEntClassifier = nltk.classify.maxent.MaxentClassifier.train(tr, 'GIS', trace=3,encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 10)"""



'"def words_features(tweet):\n    tweet_words=set(tweet)\n    features={}\n    features[word]= word in tweet_words\n    return features\n    \ndef extract_fts(x,y):\n    return[[(word_features(WordPunctTokenizer().tokenize(x[i]),y[i]) for i in range(len(x)))]]\n\ntr=extract_fts(x_tr,y_tr)\nMaxEntClassifier = nltk.classify.maxent.MaxentClassifier.train(tr, \'GIS\', trace=3,encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 10)'

In [71]:
#extract_features()
#featureList
from gensim.models import Word2Vec
import time
# Skip-gram model (sg = 1)
size = 100
window = 3
min_count = 1
workers = 3
sg = 1

word2vec_model_file = 'Word2vec_' + str(size) + '.model'
start_time = time.time()
stemmed_tokens = pd.Series(train_data['Tweet_stemmed']).values
# Train the Word2Vec Model
w2v_model = Word2Vec(stemmed_tokens, min_count = min_count, size = size, workers = workers, window = window, sg = sg)
print("Time taken to train word2vec model: " + str(time.time() - start_time))
w2v_model.save(word2vec_model_file)

from gensim.models import KeyedVectors
from gensim.models import Word2Vec
word2vec_path = 'Word2vec100.model'
word2vec = KeyedVectors.load(word2vec_path)
""""word2vec_features=[]
#for word in flat_ft:
    #word2vec_features.append(np.average(word2vec[word]))

#print(len(word2vec_features))
#word2vec_df=pd.DataFrame(word2vec_features)
#ft_sent=ft_sent.reset_index()
#w2v_s=ft_sent['sentiment'].values.tolist()
#len(w2v_s)
#word2vec_df['sentiment']=w2v_s
#word2vec_df"""

'"word2vec_features=[]\n#for word in flat_ft:\n    #word2vec_features.append(np.average(word2vec[word]))\n\n#print(len(word2vec_features))\n#word2vec_df=pd.DataFrame(word2vec_features)\n#ft_sent=ft_sent.reset_index()\n#w2v_s=ft_sent[\'sentiment\'].values.tolist()\n#len(w2v_s)\n#word2vec_df[\'sentiment\']=w2v_s\n#word2vec_df'

In [75]:
tweets=[]
featureList=[]
for index, row in train_data.iterrows(): 
    sentiment = row['sentiment']
    featureVector = row['Tweet_stemmed']
    featureList.extend(featureVector)
    tweets.append((featureVector, sentiment));

# print(ex)

featureList = list(set(featureList))
#print(tweets[1][0])
# tweets_new = tweets_new[0:2]
#print(tweets_new)

In [1]:
training_set_k = [extract_features(twt) for twt in tweets]
# training_set_k

NameError: name 'tweets' is not defined

In [97]:
print(training_set_k[0][0])

{'': False, 'apologis': False, 'releas': False, 'win': False, 'urdu': False, 'shahdol': False, 'jamshedpur': False, 'loos': False, 'delhir': False, 'wear': False, 'suffrng': False, 'kirantar': False, 'waghodiya': False, 'vishwa': False, 'folow': False, 'underst': False, 'hd': False, 'button': False, 'region': False, 'scum': False, 'quick': False, 'andhrapradeshcm': False, 'appear': False, 'levi': False, 'action': False, 'httpstcouthwwqpmm': False, 'hail': False, 'hdfcnrebankingfail': False, 'grp': False, 'loan': False, 'httpstcoknfzcvtu': False, 'gain': False, 'branchmanag': False, 'ghosh': False, 'ten': False, 'uncop': False, 'withdraw': False, 'aw': False, 'nhi': False, 'httpstcodvusdnpn': False, 'nandnagri': False, 'eftchequ': False, 'move': False, 'balancewhi': False, 'tuticorin': False, 'lumpsom': False, 'effortless': False, 'breakup': False, 'bahraich': False, 'corner': False, 'jock': False, 'dissapoint': False, 'school': False, 'splli': False, 'bads': False, 'aprciat': False, 'n

In [ ]:
#Max Entropy Classifier
MaxEntClassifier = nltk.classify.maxent.MaxentClassifier.train(training_set_k, 'GIS', trace=3,encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 3)

  ==> Training (3 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------


In [ ]:
#Max Entropy Classifier
MaxEntClassifier = nltk.classify.maxent.MaxentClassifier.train(training_set_k, 'GIS', trace=3,encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 3)

In [127]:
test_data = pd.concat([hdfc_test,axis_test,sbi_test,kotak_test,icici_test])
test_data

,tweet,sentiment,bank,@,Tweet_str,Tweet_cleaned,Tweet_stemmed,Tweet_lemmatized_stemmed
2,@HDFCBank_Cares Both the ATM machines here in ...,displeasure,[hdfc],"[@HDFCBank_Cares, @sanatm]",hdfcbankcares both the atm machines here in ti...,"[hdfcbankcares, atm, machines, times, india, n...","[hdfcbankcar, atm, machin, time, india, noida,...","[hdfcbankcar, atm, machin, time, india, noida,..."
9,@HDFC_Bank @HDFCBank_Cares received an email s...,compliment,"[hdfc, hdfc, hdfc]","[@HDFC_Bank, @HDFCBank_Cares]",hdfcbank hdfcbankcares received an email sayin...,"[hdfcbank, hdfcbankcares, received, email, say...","[hdfcbank, hdfcbankcar, receiv, email, say, at...","[hdfcbank, hdfcbankcar, receiv, email, say, at..."
11,"@HDFCBank_Cares the ATM at Sector-8, Charkop, ...",displeasure,[hdfc],[@HDFCBank_Cares],hdfcbankcares the atm at sector charkop kandiv...,"[hdfcbankcares, atm, sector, charkop, kandival...","[hdfcbankcar, atm, sector, charkop, kandivaliw...","[hdfcbankcar, atm, sector, charkop, kandivaliw..."
16,@ICICIBank_Care @HDFCBank_Cares @AxisBank #nom...,displeasure,"[icici, hdfc, axis]","[@ICICIBank_Care, @HDFCBank_Cares, @AxisBank, ...",icicibankcare hdfcbankcares axisbank nomoney i...,"[icicibankcare, hdfcbankcares, axisbank, nomon...","[icicibankcar, hdfcbankcar, axisbank, nomoney,...","[icicibankcar, hdfcbankcar, axisbank, nomoney,..."
20,RT @Tarun260: Thanks to @HDFC_Bank @HDFCBank_C...,compliment,"[hdfc, hdfc]","[@Tarun260, @HDFC_Bank, @HDFCBank_Cares]",rt tarun thanks to hdfcbank hdfcbankcares mgmt...,"[rt, tarun, thanks, hdfcbank, hdfcbankcares, m...","[rt, tarun, thank, hdfcbank, hdfcbankcar, mgmt...","[rt, tarun, thank, hdfcbank, hdfcbankcar, mgmt..."
...,...,...,...,...,...,...,...,...
2616,@ICICIBank_Care thanks. ATMs were working today.,compliment,[icici],[@ICICIBank_Care],icicibankcare thanks atms were working today,"[icicibankcare, thanks, atms, working, today]","[icicibankcar, thank, atm, work, today]","[icicibankcar, thank, atm, work, today]"
2618,@ICICIBank_Care your Patna zonal office car lo...,displeasure,[icici],[@ICICIBank_Care],icicibankcare your patna zonal office car loan...,"[icicibankcare, patna, zonal, office, car, loa...","[icicibankcar, patna, zonal, offic, car, loan,...","[icicibankcar, patna, zonal, offic, car, loan,..."
2620,Unfortunately 70% in queue do not need cash @H...,miscellaneous,"[hdfc, icici]","[@HDFC_Bank, @ICICIBank_Care, @idea_cares, @Vo...",unfortunately in queue do not need cash hdfcb...,"[unfortunately, queue, need, cash, hdfcbank, i...","[unfortun, queue, need, cash, hdfcbank, icicib...","[unfortun, queue, need, cash, hdfcbank, icicib..."
2623,@ICICIBank_Care #chandakochar @ICICIBank dahis...,compliment,"[icici, icici]","[@ICICIBank_Care, @ICICIBank, @PMOIndia]",icicibankcare chandakochar icicibank dahisar w...,"[icicibankcare, chandakochar, icicibank, dahis...","[icicibankcar, chandakochar, icicibank, dahisa...","[icicibankcar, chandakochar, icicibank, dahisa..."


In [ ]:
classifier_f = open("maxent.pickle", "rb") #loading the saved classifier
classifier = pickle.load(classifier_f)
classifier_f.close()

In [105]:
''''def dict
test_data['test_set_formatted'] = (test_data['Tweet_stemmed']).apply(lambda x: list_to_dict(x))
test_data_formatted=test_data['test_set_formatted']
res=[]
test_data_formatted=test_data_formatted.to_frame()
test_set_formatted=test_data_formatted.values.tolist()

''for index,row in test_data_formatted.iterrows():
    maxent_pred=classifier.classify(row['test_set_formatted'])
    res.append(maxent_pred)

""""for tweet in test_set_formatted:
        text = tweet[0]
        label = classifier.classify(extract_features(text))
        results.append( label)"""
        
classifier.classify(extract_features(test_set_formatted[0]))'''
        
test_data_final=test_data[['Tweet_stemmed','sentiment']]
test_data_final=test_data_final.values.tolist()


In [113]:
def list_to_dict(words_list):
    return dict([(word, True) for word in words_list])
test_set_formatted = [(list_to_dict(element[0]), element[1]) for element in test_data_final]
#test_set_formatted
tweet_id=0
for review in test_set_formatted:
        text = review[0]
        label = MaxEntClassifier.classify(text)
        results.append((str(tweet_id), label))
        tweet_id += int(1)
#test_set_formatted = [(list_to_dict(element[0]), element[1]) for element in test_data]
results

['miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscellaneous',
 'miscella